In [1]:
#Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
import re
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium.common.exceptions import NoSuchElementException



In [2]:
# Carga archivo patri
df_pelis = pd.read_csv("peliculas_api.csv")

# los ID son string
df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str)

# abre google
driver = webdriver.Chrome()

# maximiza
driver.maximize_window()

# Itera sobre los ID
for id_imdb in df_pelis["ID de la Película"][:1]:
    try:
        # carga la página de IMDB
        url = f"https://www.imdb.com/title/{id_imdb}/"
        driver.get(url)

        sleep(3)

        # acepta las cookies
        try:
            driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        except NoSuchElementException:
            pass   

        # Haz clic en el enlace "Reparto y equipo"
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button').click()
        sleep(2)

         #Voy a reparto y equipo
        driver.find_element("xpath", "/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a").click()
        sleep(2)

        # Inicializa una lista para almacenar las tuplas de actores
        lista_actores = []

        # itera sobre los 10 primeros actores
        for i in range(2, 12):
            try:
                # Haz clic en el enlace del actor
                driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
                sleep(3)

                # nombre actr
                nombre_act = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > h1 > span").text

                #Extraemos las pelis por las que se le conoce
                conocido_por1 = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
                conocido_por2 = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
                conocido_por = [conocido_por1, conocido_por2] if conocido_por2 != "N/A" else [conocido_por1]

                # año de nacimiento del actor 
                try:        
                  año_nacimiento_texto = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-4.eqrmzR > div.sc-92625f35-6.cApiRN > div.sc-92625f35-11.qOBXf > section > aside > div > span:nth-child(2)").text
                  # Extraer solo el año
                  año_nacimiento = año_nacimiento_texto.split()[-1]           
                except NoSuchElementException:
                    try:
                     año_nacimiento= driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-4.brwmyD > div > div.sc-92625f35-11.qOBXf > section > aside > div > span:nth-child(2)").text
                     # Extraer solo el año
                     año_nacimiento = año_nacimiento_texto.split()[-1]   
                    except NoSuchElementException:
                       año_nacimiento= "N/A"
                # la profesión del actor
                profesiones = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text.split("\n")
                sleep(2)
                # los premios del actor, si los tiene
                try:
                    premios = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text
                except NoSuchElementException:
                    premios = "N/A"

                # Crea la tupla de actor y agrega a la lista
                actor_info = (nombre_act, año_nacimiento, conocido_por, profesiones, premios)
                lista_actores.append(actor_info)

            except NoSuchElementException:
                pass  # No se encontró el enlace del actor, continuar con el siguiente

            finally:
                # Regresa a la página anterior para seleccionar al siguiente actor
                driver.back()
                sleep(2)

        # Imprime los datos de los actores en forma de lista de tuplas
        print("Datos de los actores:")
        print(lista_actores)

    except Exception as e:
        print(f"Error al procesar la película con ID de IMDb {id_imdb}: {e}")

# Cierra el WebDriver al finalizar
driver.quit()


Datos de los actores:
[('Éva Gábor', 'N/A', ['Europa, East', 'Pál Adrienn'], ['Reparto'], 'N/A'), ('István Znamenák', '1966', ['Az állampolgár', 'Taxidermia'], ['Reparto', 'Dirección', 'Dirección de fotografía y cámara'], 'N/A'), ('Ákos Horváth', '1968', ['Kontroll', 'Haunting Trophies'], ['Reparto', 'Guion'], 'N/A'), ('Lia Pokorny', 'N/A', ['Ízig-vérig', 'Well'], ['Reparto'], 'N/A'), ('Izabella Hegyi', 'N/A', ['Friss levegö', 'Pál Adrienn'], ['Reparto'], 'N/A'), ('Szabolcs Thuróczy', 'N/A', ['Matones sobre ruedas', 'Sandwich'], ['Reparto', 'Guion'], '3 premios'), ('Ági Margitai', 'N/A', ['Ilusiones de un mentiroso', 'Ripacsok'], ['Reparto'], 'N/A'), ('Eszter Márton', '1968', ['Nunca moriremos (Sose halunk meg)', 'Jóban Rosszban'], ['Guion', 'Reparto'], 'N/A'), ('Lóránt Matta', '1968', ['Pál Adrienn', 'Mi kérünk elnézést!'], ['Reparto'], 'N/A'), ('Róbert Kardos', '1968', ['Last Chance', 'Papsajt'], ['Reparto', 'Dirección', 'Guion'], 'N/A')]


In [3]:
df_actores = pd.DataFrame(lista_actores, columns=["Nombre", "Año de nacimiento", "Películas conocidas", "Profesiones", "Premios"])

# Imprimir el DataFrame
print("DataFrame de actores:")
print(df_actores)

DataFrame de actores:
              Nombre Año de nacimiento  \
0          Éva Gábor               N/A   
1    István Znamenák              1966   
2       Ákos Horváth              1968   
3        Lia Pokorny               N/A   
4     Izabella Hegyi               N/A   
5  Szabolcs Thuróczy               N/A   
6       Ági Margitai               N/A   
7      Eszter Márton              1968   
8       Lóránt Matta              1968   
9      Róbert Kardos              1968   

                                 Películas conocidas  \
0                        [Europa, East, Pál Adrienn]   
1                       [Az állampolgár, Taxidermia]   
2                      [Kontroll, Haunting Trophies]   
3                                 [Ízig-vérig, Well]   
4                        [Friss levegö, Pál Adrienn]   
5                   [Matones sobre ruedas, Sandwich]   
6              [Ilusiones de un mentiroso, Ripacsok]   
7  [Nunca moriremos (Sose halunk meg), Jóban Ross...   
8          